In [1]:
!pip install scikeras

In [2]:
import pandas as pd

data = pd.read_csv('../../data/processed/cleaned_heart_data.csv')

In [3]:
from sklearn.model_selection import train_test_split

seed_value = 42
# Separate features (X) and target (y)
y = data['HeartDisease']
X = data.drop('HeartDisease', axis=1) # Drop target column

# Since the data is pre-processed we don't need to do any further processing but simply split it
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed_value)

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input, BatchNormalization
from keras.regularizers import l2

# MLP model creation
def create_model(neurons=64, activation='relu', optimizer='adam', input_shape=(X_train.shape[1],)):
    model = Sequential([
        Input(shape=input_shape),
        Dense(neurons, activation=activation),
        BatchNormalization(),
        Dense(neurons // 2, activation=activation), # Half neurons in the second layer
        Dropout(0.3),
        Dense(1, activation='sigmoid') # Output layer for binary classification
    ])

    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [5]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier


# 2. Create a KerasClassifier
model = KerasClassifier(model=create_model, verbose=0)

# # 3. Define the hyperparameter grid
param_grid = {
    'model__neurons': [64, 128],
    'model__activation': ['relu', 'leaky_relu', 'tanh'],
    'optimizer': ['adam', 'rmsprop'],
    'batch_size': [16, 32],
    'epochs': [50, 20]
}

# 4. Create GridSearchCV object
grid = GridSearchCV(estimator=model,
                    param_grid=param_grid,
                    cv=3,
                    scoring='accuracy',
                    error_score='raise',
                    n_jobs=-1)

# 5. Fit the grid search to your training data
grid_result = grid.fit(X_train, y_train)

# 6. Print the best parameters and score
print(f"Best Parameters: {grid_result.best_params_}")
print(f"Best Accuracy: {grid_result.best_score_:.4f}")

Best Parameters: {'batch_size': 16, 'epochs': 50, 'model__activation': 'leaky_relu', 'model__neurons': 64, 'optimizer': 'adam'}
Best Accuracy: 0.8423


In [6]:

# 7. Evaluate the best model on the test set
best_model = grid_result.best_estimator_.model_

# loss, accuracy = best_model.model.evaluate(X_test, y_test)  # Access the underlying Keras model
best_model.summary()
loss, accuracy = best_model.evaluate(X_test, y_test)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,925 (38.77 KB)

 Trainable params: 3,265 (12.75 KB)

 Non-trainable params: 128 (512.00 B)

 Optimizer params: 6,532 (25.52 KB)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8827 - loss: 0.2997 
Test Loss: 0.3003
Test Accuracy: 0.8800


In [7]:
from keras.metrics import Recall

recall_metric = Recall()
y_pred = best_model.predict(X_test)
recall_value = recall_metric(y_test, y_pred)

print(f"Recall: {recall_value.numpy():.4f}")


5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Recall: 0.9367


In [23]:
best_model.save('../../models/best_complete_model.keras')  # Save the model in keras format

print("Best keras model with complete dataset saved to 'best_complete_model.keras'")


Best keras model with complete dataset saved to 'best_complete_model.keras'


In [8]:
#baseline - 'Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBP', 'RestingECG'
#stress test - 'MaxHR', 'Oldpeak', 'ExerciseAngina', 'ST_Slope'

# Baseline features
X_train_reduced = X_train.drop(columns=['MaxHR', 'Oldpeak', 'ExerciseAngina_Y', 'ST_Slope_Flat', 'ST_Slope_Up'])
X_test_reduced = X_test.drop(columns=['MaxHR', 'Oldpeak', 'ExerciseAngina_Y', 'ST_Slope_Flat', 'ST_Slope_Up'])


In [9]:
X_train_reduced.head(5)

,Age,RestingBP,Cholesterol,FastingBS,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST
70,57,140,265,0,True,True,False,False,False,True
164,52,140,225,0,False,True,False,False,True,False
710,56,140,294,0,False,True,False,False,False,False
265,54,160,305,0,True,True,False,False,True,False
250,44,135,491,0,True,False,False,False,True,False


In [10]:
X_test_reduced.head(5)

,Age,RestingBP,Cholesterol,FastingBS,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST
208,28,130,132,0,True,True,False,False,False,False
259,55,122,320,0,False,True,False,False,True,False
97,39,160,147,1,True,False,True,False,True,False
148,50,120,168,0,True,True,False,False,True,False
395,71,130,221,0,True,False,False,False,False,True


In [11]:
# Here we will do two different experiments

# 1. Re-train the best model with just the reduced data.
# for this we will create a new model using all the parameters
# found by usage of grid search but using the new data shape
best_model_reduced = create_model(
    neurons=grid_result.best_params_.get('model__neurons'),
    activation=grid_result.best_params_.get('model__activation'),
    optimizer=grid_result.best_params_.get('optimizer'),
    input_shape=(X_train_reduced.shape[1],))

best_model_reduced.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,073 (12.00 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 128 (512.00 B)

In [12]:
import numpy as np

X_train_reduced = X_train_reduced.astype(np.float32)
X_test_reduced = X_test_reduced.astype(np.float32)

In [13]:
best_model_reduced.fit(X_train_reduced,
                       y_train,
                       epochs=grid_result.best_params_.get('epochs'),
                       batch_size=grid_result.best_params_.get('batch_size'),
                       validation_data=(X_test_reduced, y_test),
                       verbose=1)

Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5185 - loss: 0.7891 - val_accuracy: 0.5133 - val_loss: 1.1278
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6501 - loss: 0.6581 - val_accuracy: 0.5200 - val_loss: 1.3874
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6699 - loss: 0.6307 - val_accuracy: 0.5400 - val_loss: 0.7907
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6394 - loss: 0.6578 - val_accuracy: 0.5333 - val_loss: 0.7681
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6934 - loss: 0.6114 - val_accuracy: 0.5667 - val_loss: 0.7314
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6960 - loss: 0.6241 - val_accuracy: 0.6000 - val_loss: 0.6623
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6290 - loss: 0.6599 - val_accuracy: 0.6067 - val_loss: 0.6599
Epoch 8/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6804 - loss: 0.6297 - val_accuracy: 0.6667 - val_loss:

In [14]:
loss, accuracy = best_model_reduced.evaluate(X_test_reduced, y_test)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7990 - loss: 0.4536
Test Loss: 0.4812
Test Accuracy: 0.7667


In [20]:
recall_metric_reduced_1 = Recall()
y_pred_reduced_1 = best_model_reduced.predict(X_test_reduced)
recall_value_reduced_1 = recall_metric_reduced_1(y_test, y_pred_reduced_1)

print(f"Recall: {recall_value_reduced_1.numpy():.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
Recall: 0.8987


In [22]:
best_model_reduced.save('../../models/best_baseline_model.keras')  # Save the model in HDF5 format

print("Best keras baseline model saved to 'best_baseline_model.keras'")


Best keras baseline model saved to 'best_baseline_model.keras'


In [16]:
model_reduced = KerasClassifier(model=create_model, verbose=0, input_shape=(X_train_reduced.shape[1],))

# 4. Create GridSearchCV object
grid_reduced = GridSearchCV(estimator=model_reduced,
                            param_grid=param_grid,
                            cv=3,
                            scoring='accuracy',
                            error_score='raise',
                            n_jobs=-1)

# 5. Fit the grid search to your training data
grid_reduced_result = grid_reduced.fit(X_train_reduced, y_train)

# 6. Print the best parameters and score
print(f"Best Parameters: {grid_reduced_result.best_params_}")
print(f"Best Accuracy: {grid_reduced_result.best_score_:.4f}")

Best Parameters: {'batch_size': 32, 'epochs': 50, 'model__activation': 'leaky_relu', 'model__neurons': 64, 'optimizer': 'adam'}
Best Accuracy: 0.7248


In [17]:
best_model_reduced_grid = grid_reduced_result.best_estimator_.model_

# loss, accuracy = best_model.model.evaluate(X_test, y_test)  # Access the underlying Keras model
best_model_reduced_grid.summary()
loss, accuracy = best_model_reduced_grid.evaluate(X_test_reduced, y_test)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,965 (35.02 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 128 (512.00 B)

 Optimizer params: 5,892 (23.02 KB)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5581 - loss: 0.9648  
Test Loss: 1.0068
Test Accuracy: 0.5467


In [19]:
recall_metric_reduced = Recall()
y_pred_reduced = best_model_reduced_grid.predict(X_test_reduced)
recall_value_reduced = recall_metric_reduced(y_test, y_pred_reduced)

print(f"Recall: {recall_value_reduced.numpy():.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Recall: 0.1392
